# 使用Claude进行分类：保险工单分类器

在本指南中，您将构建一个高精度的分类系统，将保险支持工单分类为10个类别。您将学习如何通过结合提示工程、检索增强生成（RAG）和思维链推理，将分类精度从70%逐步提升到95%以上。

在本指南结束时，您将了解如何设计能够处理复杂业务规则、在有限训练数据下工作并提供可解释结果的分类系统。

## 先决条件
- Python 3.11+ 且有基本了解
- Anthropic API密钥（[在此获取](https://console.anthropic.com)）
- VoyageAI API密钥（可选 - 嵌入已预计算）
- 理解分类问题

## 设置

首先，我们将安装所需的软件包

- anthropic 
- voyageai
- pandas
- matplotlib
- sklearn
- numpy

我们还将从环境中加载API密钥并设置我们的模型名称。

In [1]:
%%capture
!pip install -U anthropic voyageai pandas numpy matplotlib scikit-learn

In [ ]:
# 设置我们的环境
import anthropic
import os

MODEL = "claude-haiku-4-5"
client = anthropic.Anthropic(
    # 这是默认值，可以省略
    api_key=os.getenv("ANTHROPIC_API_KEY"),
)

## 使用Claude进行分类

大语言模型（LLMs）已经彻底改变了分类领域，特别是在传统机器学习系统面临挑战的领域。LLMs在处理具有复杂业务规则和低质量或有限训练数据特征的分类问题方面展现出了卓越的成功。此外，LLMs还具备为其行为提供自然语言解释和理由的能力，增强了分类过程的可解释性和透明度。通过利用LLMs的力量，我们可以构建超越传统机器学习方法的分类系统，并在数据稀缺或业务要求复杂的情境中表现出色。

在本指南中，我们将探讨如何利用LLMs来处理高级分类任务。我们将涵盖以下关键组件和步骤：

1. **数据准备**：我们将开始准备训练和测试数据。训练数据将用于构建分类模型，而测试数据将用于评估其性能。适当的数据准备对于确保分类系统的有效性至关重要。

2. **提示工程**：提示工程在利用LLMs进行分类方面发挥着至关重要的作用。我们将设计一个提示模板，定义用于分类的提示的结构和格式。提示模板将结合用户查询、类别定义和来自向量数据库的相关示例。通过精心制作提示，我们可以引导LLM生成准确且上下文相关的分类。

3. **实现检索增强生成（RAG）**：为了增强分类过程，我们将采用向量数据库来存储和高效检索训练数据的嵌入。向量数据库支持相似性搜索，使我们能够为给定查询找到最相关的示例。通过用检索到的示例增强LLM，我们可以提供额外的上下文并提高生成分类的准确性。

4. **测试和评估**：一旦我们的分类系统构建完成，我们将使用转换后的测试数据严格测试其性能。我们将迭代测试查询，使用分类函数对每个查询进行分类，并将预测类别与预期类别进行比较。通过分析分类结果，我们可以评估系统的有效性并确定改进领域。

## 问题定义：保险工单分类器

*注意：本示例中使用的问题定义、数据和标签均由Claude 3 Opus综合生成*

在保险行业中，客户支持在确保客户满意度和留存方面发挥着至关重要的作用。保险公司每天都会收到大量支持工单，涵盖广泛的主题，如账单、政策管理、索赔协助等。人工分类这些工单可能耗时且效率低下，导致响应时间延长，并可能影响客户体验。

#### 类别定义

1. 账单咨询
- 有关发票、收费、费用和保费的问题
- 要求澄清账单报表
- 有关支付方式和到期日的咨询

2. 政策管理
- 要求更改、更新或取消政策
- 有关政策续保和恢复的问题
- 有关添加或删除保险选项的咨询

3. 索赔协助
- 有关索赔流程和申请程序的问题
- 要求帮助提交索赔文件
- 有关索赔状态和支付时间线的咨询

4. 保险范围解释
- 有关特定政策类型所涵盖内容的问题
- 要求澄清保险范围和排除条款
- 有关免赔额和自付费用的咨询


5. 报价和建议书
- 要求新的政策报价和价格比较
- 有关可用折扣和捆绑选项的问题
- 有关从另一家保险公司转换的咨询


6. 账户管理
- 要求登录凭据或密码重置
- 有关在线账户功能和特性的问题
- 有关更新联系或个人信息的咨询


7. 账单争议
- 有关意外或错误收费的投诉
- 要求退款或保费调整
- 有关滞纳金或催收通知的咨询


8. 索赔争议
- 有关被拒绝或赔付不足的索赔的投诉
- 要求重新考虑索赔决定
- 有关对索赔结果提出上诉的咨询


9. 政策比较
- 有关政策选项之间差异的问题
- 要求帮助决定保险级别
- 有关政策与竞争对手产品比较的咨询


10. 一般咨询
- 有关公司联系信息或营业时间的问题
- 要求提供有关产品或服务的一般信息
- 不易归入其他类别的咨询




## 加载和准备数据

现在我们已经定义了问题和类别，让我们加载标记的训练和测试数据。我们将读取TSV文件，将它们转换为结构化格式，并准备测试集以进行评估。

训练数据包含68个标记示例，我们稍后将使用它们进行检索增强生成（RAG）。测试集也包含68个示例，我们将使用它们来评估我们的分类方法。

我们将使用以下数据集：
- `./data/test.tsv`
- `./data/train.tsv`

In [ ]:
import pandas as pd

data = {
    "train": [],
    "test": [],
}


# 辅助函数：将DataFrame转换为字典列表
def dataframe_to_dict_list(df):
    return df.apply(lambda x: {"text": x["text"], "label": x["label"]}, axis=1).tolist()


# 将TSV文件读入DataFrame
test_df = pd.read_csv("./data/test.tsv", sep="\t")
data["test"] = dataframe_to_dict_list(test_df)

train_df = pd.read_csv("./data/train.tsv", sep="\t")
data["train"] = dataframe_to_dict_list(train_df)


# 理解数据集中的标签
labels = list(set(train_df["label"].unique()))

# 打印第一个训练示例和训练示例的数量
print(data["train"][0], len(data["train"]))

# 创建测试集
X_test = [example["text"] for example in data["test"]]
y_test = [example["label"] for example in data["test"]]

# 打印测试集的长度
print(len(X_test), len(y_test))

## 评估框架

在构建分类器之前，让我们建立一个评估框架来衡量它们的性能。我们将创建两个关键函数：

1. **`evaluate(X, y, classifier, batch_size)`**：使用并发执行在所有测试示例上运行您的分类器以提高速度，然后计算准确率指标并生成混淆矩阵。`batch_size`参数控制要进行的并发API请求数量。

2. **`plot_confusion_matrix(cm, labels)`**：可视化混淆矩阵，显示哪些类别之间存在混淆，有助于识别分类器的薄弱环节。

这个框架使我们能够经验性地比较不同的分类方法，不仅了解总体准确率，还了解哪些特定类别具有挑战性。

**速率限制**：`MAXIMUM_CONCURRENT_REQUESTS`默认设置为1。如果您有更高的速率限制层级，可以增加此值以加快评估速度。查看您层级的[速率限制文档](https://docs.claude.com/en/api/rate-limits)。

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import concurrent.futures
import numpy as np

MAXIMUM_CONCURRENT_REQUESTS = 5


def plot_confusion_matrix(cm, labels):
    # 可视化混淆矩阵
    fig, ax = plt.subplots(figsize=(8, 8))
    im = ax.imshow(cm, cmap="Blues")

    # 添加颜色条
    ax.figure.colorbar(im, ax=ax)

    # 设置刻度标签和位置
    ax.set_xticks(np.arange(len(labels)))
    ax.set_yticks(np.arange(len(labels)))
    ax.set_xticklabels(labels, rotation=45, ha="right")
    ax.set_yticklabels(labels)

    # 为每个单元格添加标签
    thresh = cm.max() / 2.0
    for i in range(len(labels)):
        for j in range(len(labels)):
            ax.text(
                j,
                i,
                cm[i, j],
                ha="center",
                va="center",
                color="white" if cm[i, j] > thresh else "black",
            )

    # 设置标签和标题
    plt.xlabel("预测标签")
    plt.ylabel("真实标签")
    plt.title("混淆矩阵")
    plt.tight_layout()
    plt.show()


def evaluate(X, y, classifier, batch_size=MAXIMUM_CONCURRENT_REQUESTS):
    # 初始化列表以存储预测和真实标签
    y_true = []
    y_pred = []

    # 存储结果及其原始索引以保持顺序
    results = [None] * len(X)

    # 创建有限工作线程的ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor(max_workers=batch_size) as executor:
        # 一次提交一个任务，但执行器只会同时运行batch_size个
        future_to_index = {executor.submit(classifier, x): i for i, x in enumerate(X)}

        # 处理完成的结果
        for future in concurrent.futures.as_completed(future_to_index):
            index = future_to_index[future]
            predicted_label = future.result()
            results[index] = predicted_label

    # 按顺序提取预测和真实标签
    y_pred = results
    y_true = y

    # 规范化y_true和y_pred
    y_true = [label.strip() for label in y_true]
    y_pred = [label.strip() for label in y_pred]

    # 计算分类指标
    report = classification_report(y_true, y_pred, labels=labels, zero_division=1)
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    print(report)
    plot_confusion_matrix(cm, labels)

## 基线：随机分类器

为了建立性能基线并验证我们的评估框架工作正确，让我们从一个随机分类器开始，它为每个工单随机选择一个类别。

这为我们提供了性能的下限：任何实际的分类方法都应该显著优于随机猜测（使用10个类别应该达到大约10%的准确率）。这个基线帮助我们量化我们的上下文工程实际添加了多少价值。

In [ ]:
import random


def random_classifier(text):
    return random.choice(labels)

In [ ]:
print("在测试集上评估随机分类方法...")
evaluate(X_test, y_test, random_classifier)

正如从随机猜测中预期的那样，混淆矩阵显示预测分散在所有类别中，没有有意义的模式。对角线（正确预测）显示每个类别只有1-4个正确分类，而错误是随机分布的。

这确认了我们的基线约为10%准确率——使用10个类别的纯机会性能。任何结构化分类方法都应该显示出更强的对角线模式，表明模型实际上在学习类别区分而不是猜测。

### 简单分类测试

现在让我们使用Claude构建一个简单的分类器。

首先，我们将以XML格式对类别进行编码。这将使Claude更容易解释信息。在XML中编码信息是一种通用提示策略，更多信息请[参见此处](https://docs.claude.com/en/docs/build-with-claude/prompt-engineering/use-xml-tags)

In [7]:
import textwrap

categories = textwrap.dedent("""<category> 
    <label>Billing Inquiries</label>
    <content> Questions about invoices, charges, fees, and premiums Requests for clarification on billing statements Inquiries about payment methods and due dates 
    </content> 
</category> 
<category> 
    <label>Policy Administration</label>
    <content> Requests for policy changes, updates, or cancellations Questions about policy renewals and reinstatements Inquiries about adding or removing coverage options 
    </content> 
</category> 
<category> 
    <label>Claims Assistance</label> 
    <content> Questions about the claims process and filing procedures Requests for help with submitting claim documentation Inquiries about claim status and payout timelines 
    </content> 
</category> 
<category> 
    <label>Coverage Explanations</label> 
    <content> Questions about what is covered under specific policy types Requests for clarification on coverage limits and exclusions Inquiries about deductibles and out-of-pocket expenses 
    </content> 
</category> 
<category> 
    <label>Quotes and Proposals</label> 
    <content> Requests for new policy quotes and price comparisons Questions about available discounts and bundling options Inquiries about switching from another insurer 
    </content> 
</category> 
<category> 
    <label>Account Management</label> 
    <content> Requests for login credentials or password resets Questions about online account features and functionality Inquiries about updating contact or personal information 
    </content> 
</category> 
<category> 
    <label>Billing Disputes</label> 
    <content> Complaints about unexpected or incorrect charges Requests for refunds or premium adjustments Inquiries about late fees or collection notices 
    </content> 
</category> 
<category> 
    <label>Claims Disputes</label> 
    <content> Complaints about denied or underpaid claims Requests for reconsideration of claim decisions Inquiries about appealing a claim outcome 
    </content> 
</category> 
<category> 
    <label>Policy Comparisons</label> 
    <content> Questions about the differences between policy options Requests for help deciding between coverage levels Inquiries about how policies compare to competitors' offerings 
    </content> 
</category> 
<category> 
    <label>General Inquiries</label> 
    <content> Questions about company contact information or hours of operation Requests for general information about products or services Inquiries that don't fit neatly into other categories 
    </content> 
</category>""")

## 构建简单分类器

现在让我们构建第一个使用Claude的真实分类器。`simple_classify`函数展示了三个关键的提示工程技术：

1. **结构化提示模板**：我们以清晰的XML格式提供类别定义和支持工单，使Claude易于解析信息。

2. **使用预填充的受控输出**：通过以`<category>`开始助手响应并设置`stop_sequences=["</category>"]`，我们强制Claude只输出类别标签——没有解释或额外文本。这使得响应解析可靠且确定性。

3. **确定性分类**：设置`temperature=0.0`确保相同输入的一致预测，这对分类任务至关重要。

In [ ]:
def simple_classify(X):
    prompt = (
        textwrap.dedent("""
    您将把客户支持工单分类为以下类别之一：
    <categories>
        {{categories}}
    </categories>

    这里是客户支持工单：
    <ticket>
        {{ticket}}
    </ticket>

    仅用类别标签在类别标签之间响应。
    """)
        .replace("{{categories}}", categories)
        .replace("{{ticket}}", X)
    )
    response = client.messages.create(
        messages=[
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": "<category>"},
        ],
        stop_sequences=["</category>"],
        max_tokens=4096,
        temperature=0.0,
        model=MODEL,
    )

    # 从响应中提取结果
    result = response.content[0].text  # pyright: ignore[reportAttributeAccessIssue]
    return result.strip()

In [ ]:
print("在测试集上评估简单分类方法...")
evaluate(X_test, y_test, simple_classify)

好多了！混淆矩阵显示了强烈的对角线模式，大多数类别都达到了高精度。

这~70%的总体准确率显著优于随机猜测，但相似类别之间的混淆表明Claude需要更多上下文来进行更精细的区分。

为了解决这些混淆模式，我们将通过向Claude提供来自我们训练数据的相关示例来添加**检索增强生成（RAG）**。它的工作原理如下：

1. **嵌入训练示例**：使用VoyageAI的嵌入模型将所有68个训练工单转换为向量嵌入
2. **语义搜索**：对于每个新工单，根据余弦相似性找到5个最相似的训练示例
3. **增强提示**：在分类提示中包含这些相似示例以指导Claude

这种方法对分类特别有效，因为：
- 相似的过去示例帮助Claude区分语义接近的类别
- 少样本学习提高了准确率而无需微调
- 检索是动态的——每个查询获得最相关的示例

我们将构建一个简单的`VectorDB`类来处理嵌入存储和相似性搜索，使用[VoyageAI的嵌入模型](https://docs.claude.com/en/docs/embeddings)。

In [ ]:
import os
import numpy as np
import voyageai
import pickle
import json


class VectorDB:
    def __init__(self, api_key=None):
        if api_key is None:
            api_key = os.getenv("VOYAGE_API_KEY")
        self.client = voyageai.Client(api_key=api_key)
        self.embeddings = []
        self.metadata = []
        self.query_cache = {}
        self.db_path = "./data/vector_db.pkl"

    def load_data(self, data):
        # 检查向量数据库是否已经加载
        if self.embeddings and self.metadata:
            print("向量数据库已加载。跳过数据加载。")
            return
        # 检查vector_db.pkl是否存在
        if os.path.exists(self.db_path):
            print("从磁盘加载向量数据库。")
            self.load_db()
            return

        texts = [item["text"] for item in data]

        # 使用for循环嵌入超过128个文档
        batch_size = 128
        result = [
            self.client.embed(texts[i : i + batch_size], model="voyage-2").embeddings
            for i in range(0, len(texts), batch_size)
        ]

        # 平铺嵌入
        self.embeddings = [embedding for batch in result for embedding in batch]
        self.metadata = [item for item in data]
        self.save_db()
        # 将向量数据库保存到磁盘
        print("向量数据库已加载并保存。")

    def search(self, query, k=5, similarity_threshold=0.75):
        query_embedding = None
        if query in self.query_cache:
            query_embedding = self.query_cache[query]
        else:
            query_embedding = self.client.embed([query], model="voyage-2").embeddings[0]
            self.query_cache[query] = query_embedding

        if not self.embeddings:
            raise ValueError("向量数据库中没有加载数据。")

        similarities = np.dot(self.embeddings, query_embedding)
        top_indices = np.argsort(similarities)[::-1]
        top_examples = []

        for idx in top_indices:
            if similarities[idx] >= similarity_threshold:
                example = {
                    "metadata": self.metadata[idx],
                    "similarity": similarities[idx],
                }
                top_examples.append(example)

                if len(top_examples) >= k:
                    break
        self.save_db()
        return top_examples

    def save_db(self):
        data = {
            "embeddings": self.embeddings,
            "metadata": self.metadata,
            "query_cache": json.dumps(self.query_cache),
        }
        with open(self.db_path, "wb") as file:
            pickle.dump(data, file)

    def load_db(self):
        if not os.path.exists(self.db_path):
            raise ValueError(
                "未找到向量数据库文件。使用load_data创建新数据库。"
            )

        with open(self.db_path, "rb") as file:
            data = pickle.load(file)

        self.embeddings = data["embeddings"]
        self.metadata = data["metadata"]
        self.query_cache = json.loads(data["query_cache"])

我们可以定义向量数据库并加载我们的训练数据。

VoyageAI对没有关联信用卡的账户有每分钟3次的速率限制。为了便于演示，我们将利用缓存。

In [ ]:
vectordb = VectorDB()
vectordb.load_data(data["train"])

## RAG增强分类器

现在让我们使用检索增强生成重新构建我们的分类器。`rag_classify`函数通过以下方式增强简单分类器：

1. **检索相似示例**：对于每个工单，我们在向量数据库中搜索5个语义上最相似的训练示例
2. **格式化为少样本示例**：我们用XML格式结构化这些示例，使用`<query>`和`<label>`标签，向Claude展示相似工单是如何被分类的
3. **增强提示**：我们在要求Claude分类新工单之前将这些示例注入提示中

这种少样本学习方法通过展示具有相似语言应如何分类的具体示例，帮助Claude更好地区分相似类别。例如，如果测试工单提到"意外收费"，检索过去"账单争议"与"账单咨询"的示例有助于Claude理解细微差别。

让我们看看这在我们上一次运行中混淆的类别上提高了多少准确率。

In [ ]:
def rag_classify(X):
    rag = vectordb.search(X, 5)
    rag_string = ""
    for example in rag:
        rag_string += textwrap.dedent(f"""
        <example>
            <query>
                "{example["metadata"]["text"]}"
            </query>
            <label>
                {example["metadata"]["label"]}
            </label>
        </example>
        """)
    prompt = (
        textwrap.dedent("""
    您将把客户支持工单分类为以下类别之一：
    <categories>
        {{categories}}
    </categories>

    这里是客户支持工单：
    <ticket>
        {{ticket}}
    </ticket>

    使用以下示例帮助您对查询进行分类：
    <examples>
        {{examples}}
    </examples>

    仅用类别标签在类别标签之间响应。
    """)
        .replace("{{categories}}", categories)
        .replace("{{ticket}}", X)
        .replace("{{examples}}", rag_string)
    )
    response = client.messages.create(
        messages=[
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": "<category>"},
        ],
        stop_sequences=["</category>"],
        max_tokens=4096,
        temperature=0.0,
        model="claude-haiku-4-5",
    )

    # 从响应中提取结果
    result = response.content[0].text.strip()
    return result

In [ ]:
print("在测试集上评估RAG方法...")
evaluate(X_test, y_test, rag_classify)

## RAG结果分析

RAG将我们的准确率从~70%提升到**94%**。混淆矩阵显示出更强的对角线，大多数类别现在都达到了完美或接近完美的准确率。

即使有了相关示例，少数剩余的错误表明仍有一些边缘情况存在歧义。这就是思维链推理可以帮助Claude更仔细地思考细微差别的地方。

## RAG结合思维链推理

思维链（CoT）提示要求Claude在做出分类决策之前"大声思考"。

这种显式推理过程帮助Claude捕获在直接分类时可能错过的细微差别。例如，区分"我有问题关于我的账单"（账单咨询）和"这个收费似乎错了"（账单争议）需要理解意图和语调——这受益于逐步分析。

`rag_chain_of_thought_classify`函数添加了一个`<scratchpad>`部分，Claude在其中进行推理，然后输出最终的`<category>`。

In [ ]:
def rag_chain_of_thought_classify(X):
    rag = vectordb.search(X, 5)
    rag_string = ""
    for example in rag:
        rag_string += textwrap.dedent(f"""
        <example>
            <query>
                "{example["metadata"]["text"]}"
            </query>
            <label>
                {example["metadata"]["label"]}
            </label>
        </example>
        """)
    prompt = (
        textwrap.dedent("""
    您将把客户支持工单分类为以下类别之一：
    <categories>
        {{categories}}
    </categories>

    这里是客户支持工单：
    <ticket>
        {{ticket}}
    </ticket>

    使用以下示例帮助您对查询进行分类：
    <examples>
        {{examples}}
    </examples>

    首先您将在便签本标签中逐步思考问题。
    您应该考虑所有提供的信息并为您的分类创建一个具体论证。

    使用以下格式响应：
    <response>
        <scratchpad>您的想法和分析在这里</scratchpad>
        <category>您选择的类别标签在这里</category>
    </response>
    """)
        .replace("{{categories}}", categories)
        .replace("{{ticket}}", X)
        .replace("{{examples}}", rag_string)
    )
    response = client.messages.create(
        messages=[
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": "<response><scratchpad>"},
        ],
        stop_sequences=["</category>"],
        max_tokens=4096,
        temperature=0.0,
        model=MODEL,
    )

    # 从响应中提取结果
    result = response.content[0].text.split("<category>")[1].strip()
    return result

In [ ]:
print("在测试集上评估带有思维链的RAG方法...")
evaluate(X_test, y_test, rag_chain_of_thought_classify)

## 思维链结果分析

思维链推理将我们的准确率推至**97%**，解决了挑战先前方法的大多数边缘情况。

**逐步改进总结：**
- 随机基线：~10%准确率
- 简单分类器：~70%准确率
- RAG分类器：94%准确率  
- **RAG + 思维链：97%准确率**

通过显式推理每个分类决策，Claude能更好地区分模糊情况。

# 使用Promptfoo扩展评估

在本指南中，我们展示了**经验评估**在工程提示时的重要性。我们没有依赖直觉，而是在每一步都测量了性能：

- 随机基线：~10% → 简单：~70% → RAG：94% → RAG + CoT：97%

这种数据驱动的方法准确地揭示了哪些技术（RAG、思维链）增加了价值以及增加了多少。有关此方法的更多信息，请参阅我们的[提示工程指南](https://docs.claude.com/en/docs/prompt-engineering)。

## 超越笔记本

虽然Jupyter笔记本对于快速迭代和探索非常出色，但生产分类系统需要更强大的评估基础设施：

- **更大的测试集**：您需要评估数百或数千个示例，而不仅仅是68个
- **多种提示变体**：A/B测试不同的措辞、结构和方法
- **模型比较**：测试不同的Claude模型（Haiku vs Sonnet）或温度设置
- **回归检测**：确保提示更改不会意外损害准确率
- **版本控制**：跟踪系统演进过程中提示性能的变化

这就是专门的评估工具如[Promptfoo](https://www.promptfoo.dev/)变得必不可少的地方。

## 运行Promptfoo评估

Promptfoo是一个开源LLM评估工具包，可自动测试跨多个配置的提示。对于本指南，我们设置了Promptfoo评估来测试所有三种方法（简单、RAG、RAG w/ CoT）跨不同的温度设置。

**要运行评估：**

1. 导航到评估目录：`cd ./evaluation`
2. 按照`./evaluation/README.md`中的设置说明进行操作
3. 运行评估并返回此处分析下面的结果

结果将显示每个提示变体在整个测试集上的表现，揭示了单独笔记本测试中可能不明显模式。

In [ ]:
import json
import pandas as pd

promptfoo_results = pd.read_csv("./data/results.csv")
examples_columns = promptfoo_results.columns[2:]

number_of_providers = 5
number_of_prompts = 3

prompts = ["简单", "RAG", "RAG w/ 思维链"]

columns = ["label", "text"] + [
    json.loads(examples_columns[prompt * number_of_providers + provider])["provider"]
    + " 提示: "
    + str(prompts[prompt])
    for prompt in range(number_of_prompts)
    for provider in range(number_of_providers)
]

promptfoo_results.columns = columns

result = (
    promptfoo_results.iloc[:, 2:].astype(str).apply(lambda x: x.str.count("PASS")).sum()
    / len(promptfoo_results)
    * 100
).sort_values(ascending=False)

print(result)

## 使用Promptfoo的系统评估

上述结果展示了大规模系统提示评估的力量。虽然我们的笔记本在开发期间提供了快速迭代，但[Promptfoo](https://www.promptfoo.dev/)使我们能够跨多个维度严格测试我们的提示：

### 我们测试的内容

使用Promptfoo，我们评估了所有三种提示方法（简单、RAG、RAG w/ CoT）跨：
- **多种温度设置**（0.0, 0.2, 0.4, 0.6, 0.8）：测试确定性分类（T=0.0）是否优于略微随机化的输出
- **相同的测试集**（68个示例）：确保所有配置的公平比较
- **自动化通过/失败检查**：每个预测都会自动与真实标签进行比较

### 主要发现

Promptfoo结果证实了我们的笔记本发现，同时揭示了额外的见解：

1. **温度对CoT影响最小**：RAG w/ CoT在T=0.0、0.2和0.8上都达到了95.59%的稳定准确率，表明思维链推理稳定了输出，无论采样随机性如何

2. **RAG对温度变化稳健**：RAG性能在大多数温度设置下保持强劲（89-94%），尽管T=0.0表现最佳（94.12%）

3. **简单提示对温度不敏感**：无论温度如何，简单分类器都保持在~70%的准确率，确认没有RAG或CoT，模型主要依赖类别定义

4. **生产推荐**：使用`temperature=0.0`和RAG w/ CoT以获得最大一致性和准确率（95.59%）

要自行运行这些评估，请参阅`./evaluation/README.md`的设置说明。